# Loading Required Libraries

In [165]:
import numpy as np
import pandas as pd
import os
from random import randint
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout
from tqdm import tqdm_notebook
from keras import backend as K
import tensorflow as tf
import gc
#pd.set_option('display.max_colwidth',50)

# Reading Files

In [166]:
Train_Image_folder='./train/images/'
Train_Mask_folder='./train/masks/'
Test_Image_folder='./test/'
Train_Image_name=os.listdir(path=Train_Image_folder)
Test_Image_name=os.listdir(path=Test_Image_folder)
Train_Image_path=[]
Train_Mask_path=[]
Train_id=[]
for i in Train_Image_name:
    path1=Train_Image_folder+i
    path2=Train_Mask_folder+i
    id1=i.split(sep='.')[0]
    Train_Image_path.append(path1)
    Train_Mask_path.append(path2)
    Train_id.append(id1)
  

Test_Image_path=[]
Test_id=[]
for i in Test_Image_name:
    path=Test_Image_folder+i
    id2=i.split(sep='.')[0]
    Test_Image_path.append(path)
    Test_id.append(id2)
    
df_Train_path=pd.DataFrame({'id':Train_id,'Train_Image_path':Train_Image_path,'Train_Mask_path':Train_Mask_path})
df_Test_path=pd.DataFrame({'id':Test_id,'Test_Image_path':Test_Image_path})

df_depths=pd.read_csv('./depths.csv')
df_sub=pd.read_csv('./sample_submission.csv')
df_Train_path=df_Train_path.merge(df_depths,on='id',how='left')
df_Test_path=df_Test_path.merge(df_depths,on='id',how='left')
df_Test_path=df_sub.merge(df_Test_path,on='id',how='left')
print(df_Train_path.shape,df_Test_path.shape)
df_Train_path.head()

(4000, 4) (18000, 4)


,Train_Image_path,Train_Mask_path,id,z
0,./train/images/44381a3f55.png,./train/masks/44381a3f55.png,44381a3f55,73
1,./train/images/70db04a203.png,./train/masks/70db04a203.png,70db04a203,776
2,./train/images/42c563d895.png,./train/masks/42c563d895.png,42c563d895,505
3,./train/images/d422a9eb8f.png,./train/masks/d422a9eb8f.png,d422a9eb8f,662
4,./train/images/6d4fca6a35.png,./train/masks/6d4fca6a35.png,6d4fca6a35,443


In [167]:
df_Test_path.drop('rle_mask',axis=1,inplace=True)
df_Test_path.head()

,id,Test_Image_path,z
0,155410d6fa,./test/155410d6fa.png,559
1,78b32781d1,./test/78b32781d1.png,298
2,63db2a476a,./test/63db2a476a.png,392
3,17bfcdb967,./test/17bfcdb967.png,698
4,7ea0fd3c88,./test/7ea0fd3c88.png,837


In [168]:
df_Train_path["images"] = [np.array(load_img(path=idx, grayscale=True)) / 255 for idx in tqdm_notebook(df_Train_path.Train_Image_path)]
df_Train_path["masks"]=[np.array(load_img(path=idx, grayscale=True)) / 255 for idx in tqdm_notebook(df_Train_path.Train_Mask_path)]
df_Train_path["coverage"] = df_Train_path.masks.map(np.sum) / pow(101, 2)
def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
df_Train_path["coverage_class"] = df_Train_path.coverage.map(cov_to_class)
#df_Train_path.head(2)

# Splitting

In [169]:
img_size_ori = 101
img_size_target = 128

def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)

def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)

In [170]:
ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = train_test_split(
    df_Train_path.id.values,
    np.array(df_Train_path.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    np.array(df_Train_path.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    df_Train_path.coverage.values,
    df_Train_path.z.values,
    test_size=0.2, stratify=df_Train_path.coverage_class, random_state=123)

In [171]:
gc.collect()

307

In [172]:
print(ids_train.shape,ids_valid.shape)
print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(cov_train.shape,cov_test.shape)
print(depth_train.shape,depth_test.shape)

(3200,) (800,)
(3200, 128, 128, 1) (3200, 128, 128, 1)
(800, 128, 128, 1) (800, 128, 128, 1)
(3200,) (800,)
(3200,) (800,)


In [173]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [174]:
# Another method
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

# Model

In [175]:
def build_model(input_layer, start_neurons):
    # 128 -> 64
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    # 64 -> 32
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    # 32 -> 16
    conv3 = Conv2D(start_neurons * 4, (5, 5), activation="relu", padding="same")(pool2)
    conv3 = Conv2D(start_neurons * 4, (5, 5), activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    # 16 -> 8
    conv4 = Conv2D(start_neurons * 8, (5, 5), activation="relu", padding="same")(pool3)
    conv4 = Conv2D(start_neurons * 8, (5, 5), activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(convm)

    # 8 -> 16
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    #uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)

    # 16 -> 32
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)

    # 32 -> 64
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)

    # 64 -> 128
    deconv1 = Conv2DTranspose(start_neurons * 1, (3,3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3,3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3,3), activation="relu", padding="same")(uconv1)

    uncov1 = Dropout(0.5)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    return output_layer

input_layer = Input((img_size_target, img_size_target, 1))
output_layer = build_model(input_layer, 16)

In [176]:
model = Model(input_layer, output_layer)
model.compile(loss="binary_crossentropy", optimizer=Adam(lr=0.001), metrics=[mean_iou])
#model.summary()

In [177]:
# Data augmentation
x_train = np.append(x_train, [np.fliplr(x) for x in x_train], axis=0)
y_train = np.append(y_train, [np.fliplr(x) for x in y_train], axis=0)

In [178]:
gc.collect()

141

In [179]:
early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint("./keras.model", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)

epochs = 80
batch_size = 32

history = model.fit(x_train, y_train,
                    validation_data=[x_valid, y_valid], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr])

Train on 6400 samples, validate on 800 samples
Epoch 1/80
6400/6400 [==============================] - 995s 155ms/step - loss: 0.5608 - mean_iou: 0.3807 - val_loss: 0.5065 - val_mean_iou: 0.3819

Epoch 00001: val_loss improved from inf to 0.50646, saving model to ./keras.model
Epoch 2/80
6400/6400 [==============================] - 963s 151ms/step - loss: 0.4569 - mean_iou: 0.3887 - val_loss: 0.4119 - val_mean_iou: 0.4127

Epoch 00002: val_loss improved from 0.50646 to 0.41194, saving model to ./keras.model
Epoch 3/80
6400/6400 [==============================] - 985s 154ms/step - loss: 0.3823 - mean_iou: 0.4414 - val_loss: 0.3451 - val_mean_iou: 0.4651

Epoch 00003: val_loss improved from 0.41194 to 0.34510, saving model to ./keras.model
Epoch 4/80
6400/6400 [==============================] - 1004s 157ms/step - loss: 0.3505 - mean_iou: 0.4832 - val_loss: 0.2937 - val_mean_iou: 0.5025

Epoch 00004: val_loss improved from 0.34510 to 0.29372, saving model to ./keras.model
Epoch 5/80
6400/

6400/6400 [==============================] - 1073s 168ms/step - loss: 0.1116 - mean_iou: 0.7694 - val_loss: 0.1794 - val_mean_iou: 0.7705

Epoch 00040: val_loss did not improve
Epoch 41/80
2880/6400 [============>.................] - ETA: 8:58 - loss: 0.1088 - mean_iou: 0.7710

KeyboardInterrupt: 

In [180]:
model = load_model("./keras.model",custom_objects={'mean_iou': mean_iou})

In [181]:
preds_valid = model.predict(x_valid).reshape(-1, img_size_target, img_size_target)
preds_valid = np.array([downsample(x) for x in preds_valid])
y_valid = np.array([downsample(x) for x in y_valid])

In [182]:
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.mean(metric)

In [183]:
thresholds = np.linspace(0, 1, 50)
ious = np.array([iou_metric_batch(y_valid, np.int32(preds_valid > threshold)) for threshold in tqdm_notebook(thresholds)])

In [184]:
threshold_best_index = np.argmax(ious[9:-10]) + 9
iou_best = ious[threshold_best_index]
threshold_best = thresholds[threshold_best_index]
print('Best Threshold: ',threshold_best)

Best Threshold:  0.571428571429


In [185]:
#threshold_best=6.5

In [186]:
# Source https://www.kaggle.com/bguberfain/unet-with-depth
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

In [187]:
x_test = np.array([upsample(np.array(load_img(path=idx, grayscale=True))) / 255 for idx in tqdm_notebook(df_Test_path.Test_Image_path)]).reshape(-1, img_size_target, img_size_target, 1)

In [188]:
preds_test = model.predict(x_test)

In [189]:
pred_dict = {idx: RLenc(np.round(downsample(preds_test[i]) > threshold_best)) for i, idx in enumerate(tqdm_notebook(df_Test_path.id.values))}

In [190]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')

In [191]:
sub.head()

,rle_mask
id,
155410d6fa,1 1007 1011 97 1112 96 1213 95 1314 94 1415 93...
78b32781d1,59 43 159 44 259 45 359 46 458 48 558 49 657 5...
63db2a476a,5956 3 6055 5 6154 8 6254 9 6354 10 6454 11 65...
17bfcdb967,4547 43 4621 5 4639 7 4647 54 4714 5488
7ea0fd3c88,
